In [521]:
import datetime
import functools
import itertools
import re


In [542]:
test_inputs = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""
lines = test_inputs.split('\n')

In [546]:
lines = [line.strip() for line in open('AoC_2023-12-12_input')]
print(len(lines))

1000


In [266]:
# part 1
start_time = datetime.datetime.now()
total = 0
for line in lines[:100]:
    springs, counts = line.split()
    counts = [int(count) for count in counts.split(',')]   
    print(springs, counts)
    groups = [group for group in springs.split('.') if group]
#     print(groups)
    
    possibles = []
    for group in groups:
        possible = []
        count = group.count('?')
#         print(group, count)
        for replacements in itertools.product('.#', repeat=count):
            new_group = group
            for replacement in replacements:
                new_group = new_group.replace('?', replacement, 1)
            
#             print(new_group)
            possible.append(new_group + '.')
            
#         print(possible)    
        possibles.append(possible)
#     print([len(possible) for possible in possibles])
        
    to_test = possibles[0]
    for i in range(1, len(possibles)):
        to_test = itertools.product(to_test, possibles[i])
        to_test = [list(entry[0]) + [entry[1]] for entry in to_test]
    to_test = [''.join(test) for test in to_test]     
    
#     print(len(to_test))
    
    n_valid = 0
    for test in to_test:
        test_counts = [len(spring) for spring in test.split('.') if spring]
        if test_counts == counts:
#             print(test, test_counts)
            n_valid += 1
            
    print(n_valid)
    total += n_valid
#     print()
    
print(total)
print(datetime.datetime.now() - start_time)

???.### [1, 1, 3]
1
.??..??...?##. [1, 1, 3]
4
?#?#?#?#?#?#?#? [1, 3, 1, 6]
1
????.#...#... [4, 1, 1]
1
????.######..#####. [1, 6, 5]
4
?###???????? [3, 2, 1]
10
21
0:00:00.005651


In [547]:
# part 2

# based on https://github.com/ekimekim/advent-2020/blob/master/2023/12.2.py
@functools.cache
def calc(springs, counts, current=0):
    
    count_index = 0
    for i, c in enumerate(springs):
        
        if c == '#':
            current += 1            
            if count_index == len(counts):
                return 0            
            if current > counts[count_index]:
                return 0           
            continue           
            
        elif c == '.':            
            if current == 0:
                continue            
            if count_index == len(counts):
                return 0                        
            if current != counts[count_index]:
                return 0
            
            current = 0
            count_index += 1
            continue
        
        else:
            new_springs = springs[i + 1:]
            new_counts = counts[count_index:]
            return calc('#' + new_springs, new_counts, current) + calc('.' + new_springs, new_counts, current)
                
    if count_index == len(counts):
        return 1
    else:
        return 0

total = 0
for line in lines:
        
    springs, counts = line.split()
    counts = tuple([int(count) for count in counts.split(',')])

#     print(springs, counts)
    
    springs = '?'.join([springs] * 5) + '.'
    counts = counts * 5
#     print(springs, counts)

    n_valid = calc(springs, counts)
#     print(n_valid)
    total += n_valid
    
#     print()
    
print(total)

65607131946466


In [535]:
def calc_arrangements(springs, counts):
    groups = [group for group in springs.split('.') if group]
#     print(groups)
    
    possibles = []
    for group in groups:
        possible = []
        count = group.count('?')
#         print(group, count)
        for replacements in itertools.product('.#', repeat=count):
            new_group = group
            for replacement in replacements:
                new_group = new_group.replace('?', replacement, 1)
            
#             print(new_group)
            possible.append(new_group + '.')
            
#         print(possible)    
        possibles.append(possible)
#     print([len(possible) for possible in possibles])
        
    to_test = possibles[0]
    for i in range(1, len(possibles)):
        to_test = itertools.product(to_test, possibles[i])
        to_test = [list(entry[0]) + [entry[1]] for entry in to_test]
    to_test = [''.join(test)[:-1] for test in to_test]
    to_test = to_test
    
#     print(len(to_test))
    
    n_valid = 0
    for test in to_test:
        test_counts = [len(spring) for spring in test.split('.') if spring]
        if test_counts == counts:
#             print(test, test_counts)
            n_valid += 1
            
#     print(n_valid)
    return n_valid

def calc_order(springs, counts):
    
    n_valid = 0
    for replacements in itertools.product('.#', repeat=springs.count('?')):
        test = springs
        for replacement in replacements:
            test = test.replace('?', replacement, 1)
        
        test_counts = [len(spring) for spring in test.split('.') if spring]
        if test_counts == counts:
            n_valid += 1
            
    return n_valid

# inspired by ekimekim
def calc_recursive(springs, counts):
    
#     print(springs)
    n_valid = 0
    if '?' in springs:
        for c in '#.':
            n_valid += calc_recursive(springs.replace('?', c, 1), counts)    
        return n_valid
    else:
        if counts == [len(spring) for spring in springs.split('.') if spring]:
            return 1
        else:
            return 0
        
def calc_fast(springs, counts):
#     print(springs, counts)

    n_valid = 0
    if '?' in springs:
        index = springs.index('?')
        trimmed = springs[:index]
        blocks = re.findall('#+', trimmed)
        for i, block in enumerate(blocks):
            if i >= len(counts):
                return 0
            if len(block) > counts[i]:
                return 0
        
        for c in '#.':
            n_valid += calc_fast(springs.replace('?', c, 1), counts)    
        return n_valid
    else:
        if counts == tuple([len(spring) for spring in springs.split('.') if spring]):
            return 1
        else:
            return 0

@functools.cache
def calc(springs, counts, current=0):
    
    count_index = 0
    for i, c in enumerate(springs):
        
        if c == '#':
            current += 1            
            if count_index == len(counts):
                return 0            
            if current > counts[count_index]:
                return 0           
            continue           
            
        elif c == '.':            
            if current == 0:
                continue            
            if count_index == len(counts):
                return 0                        
            if current != counts[count_index]:
                return 0
            
            current = 0
            count_index += 1
            continue
        
        else:
            new_springs = springs[i + 1:]
            new_counts = counts[count_index:]
            return calc('#' + new_springs, new_counts, current) + calc('.' + new_springs, new_counts, current)
                
    if count_index == len(counts):
        return 1
    else:
        return 0

#     if counts == () and springs == '':
#             return 1 
    
#     groups = [spring for spring in springs.split('.') if spring]
#     for i, group in enumerate(groups):
        
#         if group.count('#') > counts[i]:
#             return 0 
        
#         if '#' in group and not(counts):
#             return 0
        
    
#         if '?' not in group:
            
#             if len(group) != counts[i]:
#                 return 0
            
#             return calc('.'.join(groups[i + 1:]), counts[i + 1:])
        
#         else:
    #return calc(springs.replace('?', '#', 1), counts) + calc(springs.replace('?', '.', 1), counts)
    
        
    

In [536]:
for line in lines[:1]:
    springs, counts = line.split()
    counts = tuple([int(count) for count in counts.split(',')])
    print(springs, counts)
#     print(calc(springs, counts), calc_fast(springs, counts))
    print(solve(springs + '.', counts))

???.### (1, 1, 3)
1


In [537]:
total = 0
for line in lines:
    springs, counts = line.split()
    counts = tuple([int(count) for count in counts.split(',')])
    print(springs, counts)
#     print(calc(springs, counts), calc_fast(springs, counts))
    print(solve(springs + '.', counts))
    print(calc(springs + '.', counts))
    
    
#     if calc(springs, counts) != calc_fast(springs, counts):
#         print(springs, counts)
#         print(calc(springs, counts), calc_fast(springs, counts))
#         print()
#     print(sub)
#     total += sub
    print()
    
print(total)

???.### (1, 1, 3)
1
1

.??..??...?##. (1, 1, 3)
4
4

?#?#?#?#?#?#?#? (1, 3, 1, 6)
1
1

????.#...#... (4, 1, 1)
1
1

????.######..#####. (1, 6, 5)
4
4

?###???????? (3, 2, 1)
10
10

0


In [541]:
start_time = datetime.datetime.now()

total = 0
for line in lines[:100]:
    springs, counts = line.split()
    counts = [int(count) for count in counts.split(',')]       
    total += calc_arrangements(springs, counts)
    
print(total)
print(datetime.datetime.now() - start_time) 

print()

start_time = datetime.datetime.now()

total = 0
for line in lines[:100]:
    springs, counts = line.split()
#     print(springs, counts)
    counts = [int(count) for count in counts.split(',')]       
    total += calc_order(springs, counts)
    
print(total)
print(datetime.datetime.now() - start_time) 
print()

start_time = datetime.datetime.now()

total = 0
for line in lines[:100]:
    springs, counts = line.split()
#     print(springs, counts)
    counts = [int(count) for count in counts.split(',')]       
    total += calc_recursive(springs, counts)
        
print(total)
print(datetime.datetime.now() - start_time)
print()

start_time = datetime.datetime.now()

total = 0
for line in lines[:100]:
    springs, counts = line.split()
#     print(springs, counts)
    counts = tuple([int(count) for count in counts.split(',')])
    total += calc_fast(springs, counts)
    
print(total)
print(datetime.datetime.now() - start_time) 
print()

total = 0
for line in lines[:100]:
    springs, counts = line.split()
#     print(springs, counts)
    counts = tuple([int(count) for count in counts.split(',')])
    total += calc(springs + '.', counts)
    
print(total)
print(datetime.datetime.now() - start_time) 

800
0:00:01.610641

800
0:00:01.253650

800
0:00:00.690282

800
0:00:00.277313

800
0:00:00.285194


In [284]:
start_time = datetime.datetime.now()

total = 0
for line in lines[:20]:
    springs, counts = line.split()
    counts = [int(count) for count in counts.split(',')]   
    print(springs, counts)
    
    print(calc_arrangements(springs, counts))
    
    springs = list(springs)
    
    start = 0
    found = True
    
    for count in counts:
        if not found:
            break
        for i in range(start, len(springs) - count + 1):
            if springs[i:i + count] == ['#'] * count:
                if i + count < len(springs) and springs[i + count] == '#':
                    found = False
                    break

                if i - 1 >= 0 and springs[i - 1] == '?':
                    springs[i - 1] = '.'
                if i + count < len(springs) and springs[i + count] == '?':
                    springs[i + count] = '.'
                start = i + count
                found = True
                break 
            
            if '.' not in springs[i:i + count]:
                if springs[i] == '#':
                    springs[i:i + count] = ['#'] * count
                    if i - 1 >= 0 and springs[i - 1] == '?':
                        springs[i - 1] = '.'
                    if i + count < len(springs) and springs[i + count] == '?':
                        springs[i + count] = '.'
                    start = i + count
                    found = True
                    break 
                    
#                 if (i == 0 or springs[i - 1] == '.') and (i + count == len(springs) or springs[i + count] == '.'):
#                     springs[i:i + count] = ['#'] * count
#                     start = i + count
#                     found = True
#                     break
                    
                else:
                    found = False
                    break
            
        else:
            found = False

    print(''.join(springs))
            
#     # now do the reverse
#     springs = springs[::-1]
#     counts = counts[::-1]

#     start = 0
#     found = True
    
#     for count in counts:
#         if not found:
#             break
#         for i in range(start, len(springs) - count + 1):
#             if springs[i:i + count] == ['#'] * count:
#                 if i + count < len(springs) and springs[i + count] == '#':
#                     found = False
#                     break

#                 if i - 1 >= 0 and springs[i - 1] == '?':
#                     springs[i - 1] = '.'
#                 if i + count < len(springs) and springs[i + count] == '?':
#                     springs[i + count] = '.'
#                 start = i + count
#                 found = True
#                 break 
            
#             if '.' not in springs[i:i + count]:
#                 if springs[i] == '#':
#                     springs[i:i + count] = ['#'] * count
#                     if i - 1 >= 0 and springs[i - 1] == '?':
#                         springs[i - 1] = '.'
#                     if i + count < len(springs) and springs[i + count] == '?':
#                         springs[i + count] = '.'
#                     start = i + count
#                     found = True
#                     break 
                    
#                 if (i == 0 or springs[i - 1] == '.') and (i + count == len(springs) or springs[i + count] == '.'):
#                     springs[i:i + count] = ['#'] * count
#                     start = i + count
#                     found = True
#                     break                   
            
#         else:
#             found = False    
    

#     springs = springs[::-1]
#     counts = counts[::-1]        
    
    springs = ''.join(springs)
    print(springs)
    
    n_valid = calc_arrangements(springs, counts)
    total += n_valid
    print(n_valid)
    
    print()

print(total)
print(datetime.datetime.now() - start_time)    


??????#?#?#?? [2, 2, 6]
1
??????#?#?#??
??????#?#?#??
1

#?????????#???? [1, 3, 2, 2]
17
#.????????#????
#.????????#????
17

?#??#..#?#???#??#. [5, 1, 1, 4]
1
?#??#..#?#???#??#.
?#??#..#?#???#??#.
1

#??????#???##??#???? [2, 1, 3, 9]
10
##.????#???##??#????
##.????#???##??#????
10

??.?????.? [3, 1]
4
??.?????.?
??.?????.?
4

?.?.?#?##????.?.?# [1, 1, 3, 2, 1]
2
?.?.?#?##????.?.?#
?.?.?#?##????.?.?#
2

#..?????#.???#? [1, 1, 1, 1, 4]
6
#..?????#.???#?
#..?????#.???#?
6

#?#??????#??????. [1, 1, 8, 1, 1]
1
#.#.?????#??????.
#.#.?????#??????.
1

??????.????#?#??. [2, 1, 6]
33
??????.????#?#??.
??????.????#?#??.
33

?#?????.?..??.#??. [1, 2, 1, 2, 1, 1]
4
?#?????.?..??.#??.
?#?????.?..??.#??.
4

??.#?.#???.#??? [1, 2, 1, 1, 4]
4
??.#?.#???.#???
??.#?.#???.#???
4

###?.?#??.?#?. [4, 4, 2]
2
####.?#??.?#?.
####.?#??.?#?.
2

.?...??#??? [1, 1]
4
.?...??#???
.?...??#???
4

????#.?.???#??#?#?. [2, 2, 3, 5]
1
????#.?.???#??#?#?.
????#.?.???#??#?#?.
1

.??.?#????????? [1, 2, 1, 1, 1]
61
.??.?#??

In [337]:
calc_fast('???.###????', [1, 1, 3])

calc_fast('.??..??...?##.?', [1, 1, 3])

calc_fast('?#?#?#?#?#?#?#???', [1, 3, 1, 6])

calc_fast('????.#...#...?????', [4, 1, 1])

calc_fast('????.######..#####.', [1, 6, 5])

calc_fast('?###??????????', [3, 2, 1])

calc_fast('??????????###??????????', [3, 2, 1])

calc_fast('?###?????????.', [3, 2, 1])

calc_fast('.????????.', [2, 1])

calc_fast('.???????', [2, 1])

10

In [233]:
print('.#.###.#.#?#?#?'.index('?'))
print(len('.#.###.#.#?#?#?') - 6)

10
9


In [131]:
calc_arrangements('?###????????', [3, 2, 1])
calc_arrangements('???????', [2, 1])

512
.###....##.# [3, 2, 1]
.###...##..# [3, 2, 1]
.###...##.#. [3, 2, 1]
.###..##...# [3, 2, 1]
.###..##..#. [3, 2, 1]
.###..##.#.. [3, 2, 1]
.###.##....# [3, 2, 1]
.###.##...#. [3, 2, 1]
.###.##..#.. [3, 2, 1]
.###.##.#... [3, 2, 1]
10
128
...##.# [2, 1]
..##..# [2, 1]
..##.#. [2, 1]
.##...# [2, 1]
.##..#. [2, 1]
.##.#.. [2, 1]
##....# [2, 1]
##...#. [2, 1]
##..#.. [2, 1]
##.#... [2, 1]
10


10

In [133]:
calc_arrangements('.??..??...?##.', [1, 1, 3])
calc_arrangements('.??..??.', [1, 1])


32
.#..#.### [1, 1, 3]
.#.#..### [1, 1, 3]
#...#.### [1, 1, 3]
#..#..### [1, 1, 3]
4
16
.#..# [1, 1]
.#.#. [1, 1]
#...# [1, 1]
#..#. [1, 1]
4


4